## Ejercicio 4: Mejorando la eficiencia con paralelismo
Investiga cómo asyncio puede integrarse con bibliotecas de procesamiento en paralelo como concurrent.futures para mejorar la eficiencia del servidor al manejar tareas que son intensivas en CPU.

Pasos:

- Modifica el servidor para utilizar concurrent.futures.ProcessPoolExecutor para ejecutar cálculos intensivos en paralelo.
- Crea tareas que requieran intensivo uso de CPU y envíalas al servidor.
- Observa y compara el rendimiento cuando se usan corutinas simples versus la ejecución en paralelo.

##servidor

In [1]:
import asyncio
import pickle
import nest_asyncio
import random
from concurrent.futures import ProcessPoolExecutor

nest_asyncio.apply()

# Diccionario para almacenar el estado de cada cliente
client_states = {}

# Crear un pool de procesos para ejecutar tareas en paralelo
executor = ProcessPoolExecutor()

async def handle_submit_job(reader, writer):
    client_id = id(writer)
    job_id = client_states.get(client_id, [0])[-1] + 1
    client_states[client_id] = client_states.get(client_id, []) + [job_id]
    writer.write(job_id.to_bytes(4, 'little'))
    await writer.drain()
    data_length = int.from_bytes(await reader.read(4), 'little')
    data = pickle.loads(await reader.read(data_length))
    
    # Ejecutar tarea en paralelo usando el pool de procesos
    result = await asyncio.get_running_loop().run_in_executor(executor, _process_data, data)
    client_states[client_id][-1] = result

def _process_data(data):
    # Simular una tarea intensiva en CPU
    complexity = random.randint(1, 5)
    time.sleep(complexity * 0.5)
    return sum(data)

async def handle_get_results(reader, writer):
    client_id = id(writer)
    job_id = int.from_bytes(await reader.read(4), 'little')
    result = client_states[client_id][job_id - 1]
    result_data = pickle.dumps(result)
    writer.write(len(result_data).to_bytes(4, 'little'))
    writer.write(result_data)
    await writer.drain()

async def accept_requests(reader, writer):
    op = await reader.read(1)
    if op[0] == 0:
        await handle_submit_job(reader, writer)
    elif op[0] == 1:
        await handle_get_results(reader, writer)

async def main():
    server = await asyncio.start_server(accept_requests, '127.0.0.1', 1936)
    async with server:
        await server.serve_forever()

asyncio.run(main())
